In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta, TH

In [13]:
# accessing 2020 jan 2nd (1st week expiry) dataframe
path = pd.DataFrame(glob('2022 bnf optidx\September\Expiry 22nd September\*'),columns=['location'])
path['csv filename'] = path['location'].apply(lambda x: x.split('\\')[-1])
path

,location,csv filename
0,2022 bnf optidx\September\Expiry 22nd Septembe...,BANKNIFTYWK41100CE.csv
1,2022 bnf optidx\September\Expiry 22nd Septembe...,BANKNIFTYWK41100PE.csv


In [62]:
# accessing 32200 ce dataframe
df1 = pd.read_csv(path['location'][0])
df1.rename(columns = {'date and time':'time','open':'ce_open','high':'ce_high','low':'ce_low','close':'ce_close','volume':'volume'},inplace=True)
df1['time'] = df1['time'].apply(lambda x: x[11:19])
df1

# accessing 32200 pe dataframe
df2 = pd.read_csv(path['location'][1])
df2.rename(columns = {'date and time':'time','open':'pe_open','high':'pe_high','low':'pe_low','close':'pe_close','volume':'volume'},inplace=True)
df2['time'] = df2['time'].apply(lambda x: x[11:19])
df2


,time,pe_open,pe_high,pe_low,pe_close,volume
0,09:15:00,522.00,570.10,486.80,555.15,7190
1,09:16:00,555.70,559.90,515.35,559.30,5931
2,09:17:00,565.25,603.00,565.25,573.65,7052
3,09:18:00,575.35,586.85,513.40,519.25,5030
4,09:19:00,512.95,514.65,491.55,503.25,6487
...,...,...,...,...,...,...
370,15:25:00,614.35,614.35,594.55,603.00,1195
371,15:26:00,609.15,639.80,609.15,636.75,905
372,15:27:00,635.70,639.85,618.10,631.65,716
373,15:28:00,632.00,638.00,629.70,634.00,480


In [65]:
# filtering and storing dataframe from entry time to exit time >>>
entry_time = '09:20'
exit_time = '15:10'
filt1 = (df1['time'] == '09:20:00') # & (df1['time'] == entry_time)
ce_entry_price = df1[filt1]['ce_open'].iloc[0]
filt2 = (df1['time'] == '15:10:00') # & (df1['time'] == '15:11')
s = df1[filt1].index[0]
e = df1[filt2].index[0]
ce_data = df1.iloc[s:e]
ce_data.reset_index(drop=True,inplace=True)
ce_data = ce_data[['time','ce_high','ce_close']].set_index('time')
ce_data


# filtering and storing dataframe from entry time to exit time  >>>
entry_time = '09:20'
exit_time = '15:10'
filt1 = (df2['time'] == '09:20:00') # & (df2['time'] == entry_time)
pe_entry_price = df2[filt1]['pe_open'].iloc[0]
filt2 = (df2['time'] == '15:10:00') # & (df2['time'] == '15:11')
s = df2[filt1].index[0]
e = df2[filt2].index[0]
pe_data = df2.iloc[s:e]
pe_data.reset_index(drop=True,inplace=True)
pe_data = pe_data[['time','pe_high','pe_close']].set_index('time')
pe_data

# concatinatng ce data and pe data
intraday_data = pd.concat([ce_data,pe_data],axis=1)
intraday_data['ce_pnl'] = 0
intraday_data['pe_pnl'] = 0
intraday_data.reset_index(inplace=True)
intraday_data.T.drop_duplicates().T

# stop loss for each leg
stop_loss_percentage = 20/100
ce_stop_loss = ce_entry_price + ce_entry_price * stop_loss_percentage
pe_stop_loss = pe_entry_price + pe_entry_price * stop_loss_percentage


In [67]:
ce_stop_loss_counter = 0
pe_stop_loss_counter = 0
ce_exit_time = ''
pe_exit_time = ''
ce_exit_price = 0
pe_exit_price = 0
ce_pnl = 0
pe_pnl = 0
pnl = 0

for index, row in intraday_data.iterrows():
    ce_ltp = row['ce_close']
    pe_ltp = row['pe_close']
    ce_high = row['ce_high']
    pe_high = row['pe_high']
    
    #criterias for exit
    #none of the stoplosses were hit and time limit is reached
    if (ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 0) & (row['time'] == exit_time):
        ce_pnl = ce_entry_price - ce_ltp
        pe_pnl = pe_entry_price - pe_ltp
        ce_stop_loss_counter = 1
        pe_stop_loss_counter = 1
        ce_exit_time = row['time']
        pe_exit_time = row['time']
        ce_exit_price = ce_ltp
        pe_exit_price = pe_ltp
        intraday_data.loc[index,'ce_pnl'] = ce_pnl
        intraday_data.loc[index,'pe_pnl'] = pe_pnl
        print('none of the stoplosses were hit and time limit is reached')
        pnl = ce_pnl + pe_pnl
        break

    #ce stop loss is hit and none were hit till now
    elif (ce_high >= ce_stop_loss) & (ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 0):
        ce_pnl = ce_entry_price - ce_stop_loss
        pe_pnl = pe_entry_price - pe_ltp
        ce_stop_loss_counter = 1
        ce_exit_time = row['time']
        ce_exit_price = ce_stop_loss
        intraday_data.loc[index,'ce_pnl'] = ce_pnl
        intraday_data.loc[index,'pe_pnl'] = pe_pnl
        print('ce stop loss is hit and none were hit till now')
        pnl = ce_pnl + pe_pnl

    #pe stop loss is hit and none were hit till now
    elif (pe_high >= pe_stop_loss) & (ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 0):
        ce_pnl = ce_entry_price - ce_ltp
        pe_pnl = pe_entry_price - pe_stop_loss
        pe_stop_loss_counter = 1
        pe_exit_time = row['time']
        pe_exit_price = pe_stop_loss
        intraday_data.loc[index,'ce_pnl'] = ce_pnl
        intraday_data.loc[index,'pe_pnl'] = pe_pnl
        print('pe stop loss is hit and none were hit till now')
        pnl = ce_pnl + pe_pnl

    #either ce or pe sl was hit and now other one is also hit
    elif (ce_stop_loss_counter == 1) & (pe_stop_loss_counter == 0):
        if (pe_high >= pe_stop_loss) & (row['time'] < exit_time):
            pe_pnl = pe_entry_price - pe_stop_loss
            pe_stop_loss_counter = 1
            pe_exit_price = pe_stop_loss
            pe_exit_time = row['time']
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl
            print('ce sl was hit and now other one is also hit')
            pnl = ce_pnl + pe_pnl
            break
        elif (row['time'] == exit_time):
            pe_pnl = pe_entry_price - pe_ltp
            pe_stop_loss_counter = 1
            pe_exit_price = pe_ltp
            pe_exit_time = row['time']
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl
            print('pe sl was hit and now other exited based on exit time')
            pnl = ce_pnl + pe_pnl
            break
        else:
            pe_pnl = pe_entry_price - pe_ltp
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl

    #either ce or pe sl was hit and now other one is also hit
    elif (ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 1):
        if (ce_high >= ce_stop_loss) & (row['time'] < exit_time):
            ce_pnl = ce_entry_price - ce_stop_loss
            ce_stop_loss_counter = 1
            ce_exit_price = ce_stop_loss
            ce_exit_time = row['time']
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl
            print('pe sl was hit and now other one is also hit')
            pnl = ce_pnl + pe_pnl
            break
        elif (row['time'] == exit_time):
            ce_pnl = ce_entry_price - ce_ltp
            ce_stop_loss_counter = 1
            ce_exit_price = ce_ltp
            ce_exit_time = row['time']
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl
            print('ce sl was hit and now other exited based on exit time')
            pnl = ce_pnl + pe_pnl
            break
        else:
            ce_pnl = ce_entry_price - ce_ltp
            intraday_data.loc[index,'ce_pnl'] = ce_pnl
            intraday_data.loc[index,'pe_pnl'] = pe_pnl

    #update the pnl during the normal course of the day
    elif (((ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 0)) | ((ce_stop_loss_counter == 1) & (pe_stop_loss_counter == 0))  | ((ce_stop_loss_counter == 0) & (pe_stop_loss_counter == 1)) | ((ce_stop_loss_counter == 1) & (pe_stop_loss_counter == 1))) or row['time'] <= exit_time:
        ce_pnl = ce_entry_price - ce_ltp
        pe_pnl = pe_entry_price - pe_ltp
        intraday_data.loc[index,'ce_pnl'] = ce_pnl
        intraday_data.loc[index,'pe_pnl'] = pe_pnl
        pnl = ce_pnl + pe_pnl

        
pd.set_option('display.max_rows',351)

intraday_data

pe stop loss is hit and none were hit till now


,time,ce_high,ce_close,pe_high,pe_close,ce_pnl,pe_pnl
0,09:20:00,513.30,473.30,532.85,521.40,26.35,-22.50
1,09:21:00,470.35,444.30,548.65,537.75,55.35,-38.85
2,09:22:00,459.55,446.90,544.05,530.00,52.75,-31.10
3,09:23:00,483.55,480.50,532.75,494.70,19.15,4.20
4,09:24:00,489.90,460.85,511.35,505.90,38.80,-7.00
5,09:25:00,474.30,464.20,530.00,505.95,35.45,-7.05
6,09:26:00,513.60,496.65,508.40,480.05,3.00,18.85
7,09:27:00,515.00,480.80,489.65,483.35,18.85,15.55
8,09:28:00,491.40,485.70,516.45,482.20,13.95,16.70
9,09:29:00,489.45,473.00,496.35,493.05,26.65,5.85


In [68]:

realised = pnl * 25
realised

564.2499999999998